In [5]:
import tkinter as tk
import random
import time
import math

# =========================
# Photon Trails Game
# =========================
# Controls:
#   Arrow keys / WASD = Move
#   P = Pause / Resume
#   R = Restart
# Collect yellow orbs, avoid your trail and red sentinels.

WIDTH, HEIGHT = 800, 500
PLAYER_RADIUS = 10
PLAYER_SPEED = 4
TRAIL_STEP = 6
TRAIL_THICKNESS = 4
TRAIL_MAX_LEN = 1000
ORB_RADIUS = 8
SENTINEL_RADIUS = 10
SENTINEL_COUNT = 3

class Orb:
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.id = None

class Sentinel:
    def __init__(self):
        self.x = random.randint(40, WIDTH-40)
        self.y = random.randint(40, HEIGHT-40)
        ang = random.uniform(0, 2*math.pi)
        spd = random.uniform(1.5, 2.5)
        self.vx, self.vy = math.cos(ang)*spd, math.sin(ang)*spd
        self.id = None

    def step(self):
        self.x += self.vx
        self.y += self.vy
        if self.x <= 10 or self.x >= WIDTH-10: self.vx *= -1
        if self.y <= 10 or self.y >= HEIGHT-10: self.vy *= -1

class Game:
    def __init__(self, root):
        self.root = root
        self.root.title("Photon Trails")
        self.canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="black")
        self.canvas.pack()

        # State
        self.state = "menu"   # menu, playing, paused, gameover
        self.keys = set()

        # Entities
        self.px, self.py = WIDTH//2, HEIGHT//2
        self.vx, self.vy = 0, 0
        self.trail = []
        self.orbs = []
        self.sentinels = []
        self.score = 0
        self.best_score = 0

        # UI
        self.label = tk.Label(root, text="Score: 0", font=("Arial", 14))
        self.label.pack()

        # Events
        self.root.bind("<KeyPress>", self.on_keydown)
        self.root.bind("<KeyRelease>", self.on_keyup)
        self.canvas.bind("<Button-1>", lambda e: self.start_game())

        self.draw_menu()
        self.loop()

    # ---------- Input ----------
    def on_keydown(self, e):
        self.keys.add(e.keysym)
        if self.state == "menu" and e.keysym in ("Return", "space"):
            self.start_game()
        if self.state == "playing" and e.keysym in ("p", "P"):
            self.pause_game()
        if self.state == "paused" and e.keysym in ("p", "P"):
            self.resume_game()
        if e.keysym in ("r", "R") and self.state in ("gameover", "paused"):
            self.start_game()

    def on_keyup(self, e):
        if e.keysym in self.keys:
            self.keys.remove(e.keysym)

    # ---------- States ----------
    def start_game(self):
        self.state = "playing"
        self.px, self.py = WIDTH//2, HEIGHT//2
        self.trail.clear()
        self.orbs = [self.spawn_orb()]
        self.sentinels = [Sentinel() for _ in range(SENTINEL_COUNT)]
        self.score = 0
        self.label.config(text=f"Score: {self.score}")

    def pause_game(self):
        self.state = "paused"

    def resume_game(self):
        self.state = "playing"

    def game_over(self):
        self.state = "gameover"
        self.best_score = max(self.best_score, self.score)

    # ---------- Spawning ----------
    def spawn_orb(self):
        return Orb(random.randint(30, WIDTH-30), random.randint(30, HEIGHT-30))

    # ---------- Loop ----------
    def loop(self):
        if self.state == "playing":
            self.update()
            self.draw()
        elif self.state == "menu":
            self.draw_menu()
        elif self.state == "paused":
            self.draw_pause()
        elif self.state == "gameover":
            self.draw_gameover()

        self.root.after(30, self.loop)

    def update(self):
        # Movement
        ax, ay = 0, 0
        if "Left" in self.keys or "a" in self.keys: ax -= 1
        if "Right" in self.keys or "d" in self.keys: ax += 1
        if "Up" in self.keys or "w" in self.keys: ay -= 1
        if "Down" in self.keys or "s" in self.keys: ay += 1
        if ax and ay: ax, ay = ax*0.7, ay*0.7
        self.vx, self.vy = ax*PLAYER_SPEED, ay*PLAYER_SPEED

        self.px += self.vx
        self.py += self.vy
        self.px = max(10, min(WIDTH-10, self.px))
        self.py = max(10, min(HEIGHT-10, self.py))

        # Trail
        self.trail.append((self.px, self.py))
        if len(self.trail) > TRAIL_MAX_LEN:
            self.trail.pop(0)

        # Sentinels
        for s in self.sentinels:
            s.step()

        # Collisions
        if self.collide_trail() or self.collide_sentinel():
            self.game_over()
            return

        # Orb collision
        for orb in list(self.orbs):
            if (self.px - orb.x)**2 + (self.py - orb.y)**2 <= (PLAYER_RADIUS+ORB_RADIUS)**2:
                self.orbs.remove(orb)
                self.score += 10
                self.label.config(text=f"Score: {self.score}")
                self.trail = self.trail[50:] if len(self.trail) > 50 else []
                self.orbs.append(self.spawn_orb())

    def collide_trail(self):
        if len(self.trail) < 12: return False
        for (x, y) in self.trail[:-12]:
            if (self.px-x)**2 + (self.py-y)**2 <= (PLAYER_RADIUS+TRAIL_THICKNESS)**2:
                return True
        return False

    def collide_sentinel(self):
        for s in self.sentinels:
            if (self.px-s.x)**2 + (self.py-s.y)**2 <= (PLAYER_RADIUS+SENTINEL_RADIUS)**2:
                return True
        return False

    # ---------- Drawing ----------
    def draw(self):
        self.canvas.delete("all")
        # Trail
        for i in range(1, len(self.trail)):
            x1,y1 = self.trail[i-1]
            x2,y2 = self.trail[i]
            self.canvas.create_line(x1,y1,x2,y2,fill="cyan",width=TRAIL_THICKNESS)
        # Orbs
        for orb in self.orbs:
            self.canvas.create_oval(orb.x-ORB_RADIUS, orb.y-ORB_RADIUS,
                                    orb.x+ORB_RADIUS, orb.y+ORB_RADIUS,
                                    fill="yellow")
        # Sentinels
        for s in self.sentinels:
            self.canvas.create_oval(s.x-SENTINEL_RADIUS, s.y-SENTINEL_RADIUS,
                                    s.x+SENTINEL_RADIUS, s.y+SENTINEL_RADIUS,
                                    fill="red")
        # Player
        self.canvas.create_oval(self.px-PLAYER_RADIUS, self.py-PLAYER_RADIUS,
                                self.px+PLAYER_RADIUS, self.py+PLAYER_RADIUS,
                                fill="lime")

    def draw_menu(self):
        self.canvas.delete("all")
        self.canvas.create_text(WIDTH//2, HEIGHT//2-40, text="PHOTON TRAILS", fill="cyan", font=("Arial", 28,"bold"))
        self.canvas.create_text(WIDTH//2, HEIGHT//2+10, text="Click or Press Enter/Space to Start", fill="white", font=("Arial", 14))

    def draw_pause(self):
        self.canvas.create_text(WIDTH//2, HEIGHT//2, text="PAUSED\nPress P to Resume", fill="yellow", font=("Arial", 20))

    def draw_gameover(self):
        self.canvas.delete("all")
        self.canvas.create_text(WIDTH//2, HEIGHT//2-40, text="GAME OVER", fill="red", font=("Arial", 28,"bold"))
        self.canvas.create_text(WIDTH//2, HEIGHT//2, text=f"Score: {self.score}   Best: {self.best_score}", fill="white", font=("Arial", 16))
        self.canvas.create_text(WIDTH//2, HEIGHT//2+40, text="Click or Press R to Restart", fill="cyan", font=("Arial", 14))

def main():
    root = tk.Tk()
    Game(root)
    root.mainloop()

if __name__ == "__main__":
    main()


In [ ]:
import pygame
import sys

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 600, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Start Screen Game")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (50, 150, 255)
RED = (255, 50, 50)

# Fonts
font = pygame.font.Font(None, 50)

# Button setup
button_rect = pygame.Rect(WIDTH // 2 - 100, HEIGHT // 2 - 40, 200, 80)

def draw_start_screen():
    screen.fill(WHITE)
    pygame.draw.rect(screen, BLUE, button_rect)
    text = font.render("START", True, WHITE)
    screen.blit(text, (button_rect.x + 50, button_rect.y + 20))
    pygame.display.flip()

def game_loop():
    # Game loop
    running = True
    x, y = WIDTH // 2, HEIGHT // 2
    clock = pygame.time.Clock()

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            x -= 5
        if keys[pygame.K_RIGHT]:
            x += 5
        if keys[pygame.K_UP]:
            y -= 5
        if keys[pygame.K_DOWN]:
            y += 5

        # Draw the game
        screen.fill(WHITE)
        pygame.draw.circle(screen, RED, (x, y), 20)
        pygame.display.flip()
        clock.tick(30)

    pygame.quit()
    sys.exit()

def main():
    # Start screen loop
    in_start_screen = True
    while in_start_screen:
        draw_start_screen()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if button_rect.collidepoint(event.pos):
                    in_start_screen = False  # Exit start screen

    game_loop()

if __name__ == "__main__":
    main()


In [7]:
import tkinter as tk
import random

# Main game class
class SimpleGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Catch the Ball Game")
        
        # Canvas setup
        self.canvas = tk.Canvas(root, width=400, height=400, bg="black")
        self.canvas.pack()

        # Start button
        self.start_button = tk.Button(root, text="Start Game", command=self.start_game)
        self.start_button.pack(pady=10)

        # Game objects
        self.paddle = None
        self.ball = None
        self.ball_speed_x = 3
        self.ball_speed_y = -3
        self.game_running = False

    def start_game(self):
        # Reset game
        self.canvas.delete("all")
        self.game_running = True

        # Create paddle
        self.paddle = self.canvas.create_rectangle(160, 350, 240, 360, fill="blue")

        # Create ball
        self.ball = self.canvas.create_oval(190, 190, 210, 210, fill="red")

        # Bind movement keys
        self.root.bind("<Left>", self.move_left)
        self.root.bind("<Right>", self.move_right)

        # Start ball movement
        self.move_ball()

    def move_left(self, event):
        if self.game_running:
            self.canvas.move(self.paddle, -20, 0)

    def move_right(self, event):
        if self.game_running:
            self.canvas.move(self.paddle, 20, 0)

    def move_ball(self):
        if not self.game_running:
            return

        self.canvas.move(self.ball, self.ball_speed_x, self.ball_speed_y)
        ball_pos = self.canvas.coords(self.ball)
        paddle_pos = self.canvas.coords(self.paddle)

        # Bounce from walls
        if ball_pos[0] <= 0 or ball_pos[2] >= 400:
            self.ball_speed_x = -self.ball_speed_x
        if ball_pos[1] <= 0:
            self.ball_speed_y = -self.ball_speed_y

        # Paddle collision
        if (ball_pos[2] >= paddle_pos[0] and ball_pos[0] <= paddle_pos[2]) and (ball_pos[3] >= paddle_pos[1] and ball_pos[1] <= paddle_pos[3]):
            self.ball_speed_y = -self.ball_speed_y

        # Game over condition
        if ball_pos[3] >= 400:
            self.game_over()
            return

        self.root.after(30, self.move_ball)

    def game_over(self):
        self.game_running = False
        self.canvas.create_text(200, 200, text="GAME OVER", fill="white", font=("Arial", 20, "bold"))

# Run the game
if __name__ == "__main__":
    root = tk.Tk()
    game = SimpleGame(root)
    root.mainloop()


In [11]:
import pygame
import random
import sys

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH = 600
HEIGHT = 400

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Create screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Catch the Balls Game")

# Paddle properties
paddle_width = 100
paddle_height = 10
paddle_x = WIDTH // 2 - paddle_width // 2
paddle_y = HEIGHT - 30
paddle_speed = 7

# Ball properties
ball_radius = 10
ball_speed = 4

# Create 3 balls
balls = []
for _ in range(3):
    balls.append({
        "x": random.randint(ball_radius, WIDTH - ball_radius),
        "y": random.randint(50, HEIGHT // 2),
        "dx": random.choice([-ball_speed, ball_speed]),
        "dy": ball_speed,
        "alive": True
    })

# Font
font = pygame.font.SysFont(None, 36)

# Score
score = 0

# Game loop control
running = True
game_over = False

clock = pygame.time.Clock()

# Main game loop
while running:
    screen.fill(WHITE)

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    if not game_over:
        # Paddle movement
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and paddle_x > 0:
            paddle_x -= paddle_speed
        if keys[pygame.K_RIGHT] and paddle_x < WIDTH - paddle_width:
            paddle_x += paddle_speed

        # Update ball positions
        active_balls = 0
        for ball in balls:
            if ball["alive"]:
                active_balls += 1
                ball["x"] += ball["dx"]
                ball["y"] += ball["dy"]

                # Bounce on walls
                if ball["x"] - ball_radius <= 0 or ball["x"] + ball_radius >= WIDTH:
                    ball["dx"] *= -1
                if ball["y"] - ball_radius <= 0:
                    ball["dy"] *= -1

                # Paddle collision
                if paddle_y < ball["y"] + ball_radius < paddle_y + paddle_height and paddle_x < ball["x"] < paddle_x + paddle_width:
                    ball["dy"] *= -1
                    score += 1

                # Ball missed
                if ball["y"] > HEIGHT:
                    ball["alive"] = False

        # Draw paddle
        pygame.draw.rect(screen, BLACK, (paddle_x, paddle_y, paddle_width, paddle_height))

        # Draw balls
        for ball in balls:
            if ball["alive"]:
                pygame.draw.circle(screen, RED, (ball["x"], ball["y"]), ball_radius)

        # Draw score
        score_text = font.render("Score: " + str(score), True, BLUE)
        screen.blit(score_text, (10, 10))

        # Check game over
        if active_balls == 0:
            game_over = True
    else:
        over_text = font.render("GAME OVER! Final Score: " + str(score), True, RED)
        screen.blit(over_text, (WIDTH // 2 - 180, HEIGHT // 2 - 20))

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()


SystemExit: 

In [12]:
import pygame
import random
import sys

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 600, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Three Ball Bounce Game")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED   = (255, 0, 0)

# Fonts
font = pygame.font.SysFont("Arial", 36)

# Ball settings (3 balls)
balls = []
for _ in range(3):
    balls.append({
        "x": random.randint(50, WIDTH-50),
        "y": random.randint(50, HEIGHT-50),
        "dx": random.choice([-4, 4]),
        "dy": random.choice([-4, 4]),
        "radius": 15
    })

# Game states
START_MENU = True
RUNNING = False

# Function to draw start screen
def draw_start_screen():
    screen.fill(WHITE)
    title = font.render("Three Ball Game", True, BLACK)
    start_text = font.render("Click to Start", True, RED)
    screen.blit(title, (WIDTH//2 - title.get_width()//2, HEIGHT//3))
    screen.blit(start_text, (WIDTH//2 - start_text.get_width()//2, HEIGHT//2))
    pygame.display.flip()

# Main game loop
def main_game():
    global RUNNING
    clock = pygame.time.Clock()

    while RUNNING:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                RUNNING = False
                pygame.quit()
                sys.exit()

        # Move balls
        for ball in balls:
            ball["x"] += ball["dx"]
            ball["y"] += ball["dy"]

            # Bounce off walls
            if ball["x"] - ball["radius"] <= 0 or ball["x"] + ball["radius"] >= WIDTH:
                ball["dx"] *= -1
            if ball["y"] - ball["radius"] <= 0 or ball["y"] + ball["radius"] >= HEIGHT:
                ball["dy"] *= -1

        # Draw everything
        screen.fill(WHITE)
        for ball in balls:
            pygame.draw.circle(screen, RED, (ball["x"], ball["y"]), ball["radius"])
        pygame.display.flip()
        clock.tick(60)

# Game start loop
while True:
    if START_MENU:
        draw_start_screen()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:  # Click to start
                START_MENU = False
                RUNNING = True
                main_game()


SystemExit: 

In [13]:
import pygame
import random
import sys

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH = 600
HEIGHT = 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Catch the Ball Game")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 200, 0)

# Fonts
font = pygame.font.Font(None, 36)
big_font = pygame.font.Font(None, 64)

# Basket
basket_width = 80
basket_height = 20
basket = pygame.Rect(WIDTH // 2 - basket_width // 2, HEIGHT - 40, basket_width, basket_height)
basket_speed = 7

# Balls
ball_radius = 10
balls = []
ball_speed = 4
for _ in range(3):  # Three balls at once
    x = random.randint(ball_radius, WIDTH - ball_radius)
    y = random.randint(-150, -20)
    balls.append(pygame.Rect(x, y, ball_radius * 2, ball_radius * 2))

# Game variables
score = 0
lives = 5
game_over = False
game_started = False

# Function to reset balls
def reset_ball(ball):
    ball.x = random.randint(ball_radius, WIDTH - ball_radius)
    ball.y = random.randint(-150, -20)

# Draw text
def draw_text(text, font, color, surface, x, y):
    txt = font.render(text, True, color)
    rect = txt.get_rect(center=(x, y))
    surface.blit(txt, rect)

# Start screen
def start_screen():
    waiting = True
    while waiting:
        screen.fill(WHITE)
        draw_text("Catch the Ball", big_font, BLUE, screen, WIDTH // 2, HEIGHT // 3)
        draw_text("Press SPACE to Start", font, BLACK, screen, WIDTH // 2, HEIGHT // 2)
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                waiting = False

# Game loop
clock = pygame.time.Clock()
start_screen()  # Show start screen
game_started = True

while True:
    screen.fill(WHITE)

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    if not game_over and game_started:
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and basket.left > 0:
            basket.x -= basket_speed
        if keys[pygame.K_RIGHT] and basket.right < WIDTH:
            basket.x += basket_speed

        # Update balls
        for ball in balls:
            ball.y += ball_speed
            if ball.colliderect(basket):
                score += 1
                reset_ball(ball)
            elif ball.y > HEIGHT:
                lives -= 1
                reset_ball(ball)

        # Draw basket and balls
        pygame.draw.rect(screen, GREEN, basket)
        for ball in balls:
            pygame.draw.circle(screen, RED, (ball.x + ball_radius, ball.y + ball_radius), ball_radius)

        # Draw score and lives
        draw_text(f"Score: {score}", font, BLACK, screen, 60, 20)
        draw_text(f"Lives: {lives}", font, BLACK, screen, WIDTH - 80, 20)

        if lives <= 0:
            game_over = True

    else:
        draw_text("GAME OVER", big_font, RED, screen, WIDTH // 2, HEIGHT // 3)
        draw_text(f"Final Score: {score}", font, BLACK, screen, WIDTH // 2, HEIGHT // 2)
        draw_text("Press R to Restart or Q to Quit", font, BLACK, screen, WIDTH // 2, HEIGHT // 2 + 40)

        keys = pygame.key.get_pressed()
        if keys[pygame.K_r]:
            score = 0
            lives = 5
            game_over = False
            for ball in balls:
                reset_ball(ball)
        elif keys[pygame.K_q]:
            pygame.quit()
            sys.exit()

    pygame.display.flip()
    clock.tick(60)


SystemExit: 

In [14]:
import pygame
import sys
import random

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 600, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Catch the Ball")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED   = (255, 0, 0)
BLUE  = (0, 0, 255)
GREEN = (0, 200, 0)
DARK_GREEN = (0, 150, 0)

# Clock
clock = pygame.time.Clock()

# Basket properties
basket_width, basket_height = 100, 15
basket_x = WIDTH // 2 - basket_width // 2
basket_y = HEIGHT - basket_height - 10
basket_speed = 7

# Ball properties
ball_radius = 15
ball_speed = 4
balls = []

# Create 3 balls spaced 1/3 across screen
for i in range(3):
    x_pos = (i + 1) * WIDTH // 4
    balls.append([x_pos, random.randint(-150, -30)])  # [x, y]

# Score
score = 0
missed = 0
game_over = False
game_started = False

# Font
font = pygame.font.SysFont("Arial", 24)

def draw_text(text, size, color, x, y, center=True):
    font_obj = pygame.font.SysFont("Arial", size)
    label = font_obj.render(text, True, color)
    rect = label.get_rect()
    if center:
        rect.center = (x, y)
    else:
        rect.topleft = (x, y)
    screen.blit(label, rect)

def start_screen():
    screen.fill(WHITE)
    draw_text("Catch the Ball", 40, BLACK, WIDTH//2, HEIGHT//3)
    pygame.draw.rect(screen, GREEN, (WIDTH//2 - 80, HEIGHT//2, 160, 50), border_radius=10)
    draw_text("START", 28, WHITE, WIDTH//2, HEIGHT//2 + 25)
    pygame.display.flip()

def game_loop():
    global basket_x, score, missed, game_over, game_started, balls

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if not game_started and event.type == pygame.MOUSEBUTTONDOWN:
                mx, my = pygame.mouse.get_pos()
                if WIDTH//2 - 80 <= mx <= WIDTH//2 + 80 and HEIGHT//2 <= my <= HEIGHT//2 + 50:
                    game_started = True

        if not game_started:
            start_screen()
            continue

        if not game_over:
            keys = pygame.key.get_pressed()
            if keys[pygame.K_LEFT] and basket_x > 0:
                basket_x -= basket_speed
            if keys[pygame.K_RIGHT] and basket_x < WIDTH - basket_width:
                basket_x += basket_speed

            # Update balls
            for ball in balls:
                ball[1] += ball_speed
                if ball[1] > HEIGHT:
                    missed += 1
                    ball[1] = random.randint(-150, -30)

                # Collision with basket
                if (basket_y - ball_radius < ball[1] < basket_y + basket_height and
                        basket_x < ball[0] < basket_x + basket_width):
                    score += 1
                    ball[1] = random.randint(-150, -30)

            if missed >= 5:  # Game over condition
                game_over = True

        # Draw everything
        screen.fill(WHITE)
        pygame.draw.rect(screen, BLUE, (basket_x, basket_y, basket_width, basket_height))

        for ball in balls:
            pygame.draw.circle(screen, RED, (ball[0], ball[1]), ball_radius)

        draw_text(f"Score: {score}", 24, BLACK, 10, 10, center=False)
        draw_text(f"Missed: {missed}", 24, BLACK, 10, 40, center=False)

        if game_over:
            draw_text("GAME OVER", 40, RED, WIDTH//2, HEIGHT//2)

        pygame.display.flip()
        clock.tick(60)

# Run the game
game_loop()


SystemExit: 

In [3]:
import pygame
import random
import sys

# Initialize pygame
pygame.init()

# Screen settings
WIDTH, HEIGHT = 600, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Catch the Ball Game")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED   = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE  = (0, 0, 255)

# Clock
clock = pygame.time.Clock()
FPS = 60

# Basket settings
basket_width, basket_height = 100, 20
basket = pygame.Rect(WIDTH//2 - basket_width//2, HEIGHT-50, basket_width, basket_height)

# Ball settings
ball_radius = 15
ball_speed = 5

# Place 3 balls: spaced at 1/3 of screen width, 1/2 of screen height
balls = [
    pygame.Rect(WIDTH//6 - ball_radius, HEIGHT//4 - ball_radius, ball_radius*2, ball_radius*2),
    pygame.Rect(WIDTH//2 - ball_radius, HEIGHT//2 - ball_radius, ball_radius*2, ball_radius*2),
    pygame.Rect(5*WIDTH//6 - ball_radius, 3*HEIGHT//4 - ball_radius, ball_radius*2, ball_radius*2),
]

ball_colors = [RED, GREEN, BLUE]

# Font
font = pygame.font.SysFont(None, 36)

# Game states
game_started = False
score = 0

def draw_start_screen():
    screen.fill(WHITE)
    title = font.render("Catch the Ball Game", True, BLACK)
    start_text = font.render("Press SPACE to Start", True, BLUE)
    screen.blit(title, (WIDTH//2 - title.get_width()//2, HEIGHT//3))
    screen.blit(start_text, (WIDTH//2 - start_text.get_width()//2, HEIGHT//2))
    pygame.display.update()

def draw_game():
    screen.fill(WHITE)

    # Draw basket
    pygame.draw.rect(screen, BLACK, basket)

    # Draw balls
    for i, ball in enumerate(balls):
        pygame.draw.circle(screen, ball_colors[i], ball.center, ball_radius)

    # Draw score
    score_text = font.render(f"Score: {score}", True, BLACK)
    screen.blit(score_text, (10, 10))

    pygame.display.update()

# Game loop
running = True
while running:
    clock.tick(FPS)

    if not game_started:
        draw_start_screen()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()
            sys.exit()

        # Start the game
        if not game_started and event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
            game_started = True
            score = 0
            # Reset ball positions
            balls = [
                pygame.Rect(WIDTH//6 - ball_radius, HEIGHT//4 - ball_radius, ball_radius*2, ball_radius*2),
                pygame.Rect(WIDTH//2 - ball_radius, HEIGHT//2 - ball_radius, ball_radius*2, ball_radius*2),
                pygame.Rect(5*WIDTH//6 - ball_radius, 3*HEIGHT//4 - ball_radius, ball_radius*2, ball_radius*2),
            ]

    if game_started:
        # Basket movement
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and basket.left > 0:
            basket.move_ip(-7, 0)
        if keys[pygame.K_RIGHT] and basket.right < WIDTH:
            basket.move_ip(7, 0)

        # Move balls
        for i, ball in enumerate(balls):
            ball.move_ip(0, ball_speed)

            # If ball hits bottom, reset
            if ball.top > HEIGHT:
                ball.x = random.randint(20, WIDTH-40)
                ball.y = -ball_radius*2

            # Check collision with basket
            if basket.colliderect(ball):
                score += 1
                ball.x = random.randint(20, WIDTH-40)
                ball.y = -ball_radius*2

        draw_game()


SystemExit: 